In [1]:
import workflow.scripts.RESources as RES
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [8]:
from RES.hdf5_handler import DataHandler
import RES.visuals as vis
store=f"data/store/resources_AL.h5"
datahandler=DataHandler(store)
cells=datahandler.from_store('cells')
boundary=datahandler.from_store('boundary')

Error: Key 'cells' not found in data/store/resources_AL.h5
Error: Key 'boundary' not found in data/store/resources_AL.h5


# Set Required Args to Activate Modules

In [3]:
import workflow.scripts.RESources as RES
import pandas as pd
import geopandas as gpd

# Iterate over provinces for both solar and wind resources
# resource_types = ['wind','solar']  #
# countries=['AL'] 
# for country in countries:
#     for resource_type in resource_types:
#         required_args = {
#             "config_file_path": 'config/config.yaml',
#             "region_short_code": country,
#             "resource_type": resource_type
#         }
        
#         # Create an instance of Resources and execute the module
#         RES_module = RES.RESources_builder(**required_args)
#         # RES_module.build(select_top_sites=True,
#         #                  use_pypsa_buses=False)
        

In [4]:
required_args = {
            "config_file_path": 'config/config.yaml',
            "region_short_code": 'AL',
            "resource_type": 'solar'
        }

In [5]:
from RES.power_nodes import GridNodeLocator

power_nodes = GridNodeLocator(**required_args)

lines = power_nodes.get_OSM_grid_lines()
lines_filtered = lines[~(lines['voltage'] == '400000')]

2025-04-04 16:09:27,340 - INFO - >> Loading locally stored OSM data for 'power' from data/downloaded_data/OSM/AL_power.geojson


In [6]:
import geopandas as gpd
from shapely.geometry import Point, LineString

# Filter only LineString geometries
lines = lines_filtered[lines_filtered.geometry.type == "LineString"].copy()
lines = lines.set_geometry("geometry")
lines.crs = cells.crs

# Create centroid column once
cells["centroid"] = cells.apply(lambda row: Point(row["x"], row["y"]), axis=1)

NameError: name 'cells' is not defined

In [ ]:


def find_nearest_line_point_with_distance(point, cell_geometry, line_gdf):
    # Lines that intersect with the cell
    lines_in_cell = line_gdf[line_gdf.geometry.intersects(cell_geometry)]

    # Nearest point within cell
    if not lines_in_cell.empty:
        distances_within = lines_in_cell.geometry.apply(lambda line: point.distance(line))
        nearest_within_geom = lines_in_cell.loc[distances_within.idxmin(), "geometry"]
        within_point = nearest_within_geom.interpolate(nearest_within_geom.project(point))
        within_distance = point.distance(within_point)
    else:
        within_point = None
        within_distance = None

    # Nearest point globally
    distances_global = line_gdf.geometry.apply(lambda line: point.distance(line))
    nearest_global_geom = line_gdf.loc[distances_global.idxmin(), "geometry"]
    else_point = nearest_global_geom.interpolate(nearest_global_geom.project(point))
    else_distance = point.distance(else_point)

    return within_point, else_point, within_distance, else_distance

def choose_optimal_point(row):
    if row["within_cell_nearest"] is not None:
        return row["within_cell_nearest"], row["distance_within"]
    else:
        return row["else_nearest"], row["distance_else"]



In [ ]:
def find_nearest_single_connection_point(point, cell_geometry, line_gdf):
    """
    For a given point (centroid) and cell geometry:
    - If any lines intersect the cell, find the nearest point on them (clipped to cell).
    - Else, find the nearest point on any global line.
    Returns: (nearest_point, distance)
    """
    # 1. Try to find intersecting lines
    intersecting_lines = line_gdf[line_gdf.geometry.intersects(cell_geometry)].copy()

    if not intersecting_lines.empty:
        # Clip the intersecting lines to the cell
        intersecting_lines["geometry"] = intersecting_lines.geometry.intersection(cell_geometry)
        lines_to_search = intersecting_lines
    else:
        # Fallback: use all lines globally
        lines_to_search = line_gdf

    # 2. Find the nearest geometry among selected lines
    distances = lines_to_search.geometry.apply(lambda line: point.distance(line))
    nearest_geom = lines_to_search.loc[distances.idxmin(), "geometry"]

    # 3. Get the exact nearest point on that geometry
    nearest_point = nearest_geom.interpolate(nearest_geom.project(point))
    distance = point.distance(nearest_point)

    return nearest_point, distance




In [ ]:
# Ensure centroid is prepared once
cells["centroid"] = cells.apply(lambda row: Point(row["x"], row["y"]), axis=1)

# Apply to each row to compute the single connection point
cells[["nearest_connection_point", "nearest_distance"]] = cells.apply(
    lambda row: find_nearest_single_connection_point(row["centroid"], row["geometry"], lines),
    axis=1, result_type="expand"
)


In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Plot nearest points
import matplotlib.patheffects as pe
data=lines_filtered
# Ensure CRS
if cells.crs is None:
    cells = cells.set_crs(epsg=3587)
if data.crs is None:
    data = data.set_crs(epsg=3587)

# Validate geometries
cells = cells[cells.is_valid]
data = data[data.is_valid]

# Setup colormap
unique_voltages = sorted(data['voltage'].dropna().unique())
cmap = cm.get_cmap('tab10', len(unique_voltages))
color_map = dict(zip(unique_voltages, cmap.colors))

# Plot setup
fig, ax = plt.subplots(figsize=(12, 12))
ax.set_facecolor("None")

# plot_with_shadow(cells, ax, color="grey", alpha=0.5, edgecolor="black", linewidth=0.5)
cells.plot(ax=ax, color='grey',alpha=0.2, edgecolor="black", linewidth=1)

# Plot nearest points with glowing effect
cells.set_geometry("nearest_connection_point").plot(
    ax=ax, 
    color="red", 
    markersize=15, 
    alpha=1, 
    label="Within Grid Nearest",
    path_effects=[pe.withStroke(linewidth=5, foreground="yellow", alpha=0.2)]
)
# cells.set_geometry("else_nearest").plot(
#     ax=ax, 
#     color="navy", 
#     markersize=15, 
#     alpha=0.3, 
#     label="Else Nearest Points",
#     path_effects=[pe.withStroke(linewidth=5, foreground="cyan", alpha=0.4)]
# )

# Plot data points with voltage colors
for voltage_level in unique_voltages:
    subset = data[data['voltage'] == voltage_level]
    subset.plot(ax=ax, color=color_map[voltage_level], markersize=50, label=f'Line (v): {voltage_level}', alpha=0.3, edgecolor='white', linewidth=2)
    

# Add clean basemap (no labels for a cleaner look)
# ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.1)

# Plot aesthetics
ax.set_title("Albania", fontsize=16, fontweight='bold', pad=15)
ax.axis("off")

# Custom legend (remove duplicates)
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(0.8, 1), frameon=False, fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
# RES_module.datahandler.to_store(RES_module.gridcells.resource_grid_cells,'cells')

In [ ]:
# RES_module.gridcells.resource_grid_cells

In [ ]:
# datahandler.from_store('cells')

In [ ]:
RES_module.gridcells.region_boundary

In [ ]:
A=RES_module.find_grid_nodes()

In [ ]:
# RES_module.build()

In [ ]:
cells=datahandler.from_store('cells')
boundary=datahandler.from_store('boundary')
solar_clusters=datahandler.from_store('clusters/solar')
wind_clusters=datahandler.from_store('clusters/wind')
solar_clusters_ts=datahandler.from_store('timeseries/clusters/solar')
wind_clusters_ts=datahandler.from_store('timeseries/clusters/wind')

In [ ]:
resource_clusters_solar,cluster_timeseries_solar=RES_module.select_top_sites(solar_clusters,
                                                                solar_clusters_ts,
                                                                    resource_max_capacity=10)

resource_clusters_wind,cluster_timeseries_solar=RES_module.select_top_sites(wind_clusters,
                                                                wind_clusters_ts,
                                                                    resource_max_capacity=20)

In [ ]:
# resource_clusters_solar.plot('potential_capacity',legend=True)
# resource_clusters_wind.plot('potential_capacity',legend=True)

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
legend_x_ax_offset=1
# Ensure 'Region' is in the columns for both boundary and cells
if 'Region' not in boundary.columns:
    boundary = boundary.reset_index(inplace=True)

# Assign a number to each region
boundary['Region_Number'] = range(1, len(boundary) + 1)

# Define custom bins and labels for solar and wind capacity
solar_bins = [0, 100, 200, 300, 500, float('inf')]  # Custom ranges
solar_labels = ['<100','100-200', '200-300', '300-500','>500']  # Labels for legend

# Define custom bins and labels for solar and wind capacity
wind_bins = [0, 300, 500, 1000, 2000,3000, float('inf')]  # Custom ranges
wind_labels = ['<300','300-500', '500-1000', '1000-2000','2000-3000', '>3000']  # Labels for legend

# Categorize potential_capacity_solar and potential_capacity_wind into bins
resource_clusters_solar['solar_category'] = pd.cut(resource_clusters_solar['potential_capacity'], bins=solar_bins, labels=solar_labels, include_lowest=True)
resource_clusters_wind['wind_category'] = pd.cut(resource_clusters_wind['potential_capacity'], bins=wind_bins, labels=wind_labels, include_lowest=True)

# Create figure and axes for side-by-side plotting
fig, (ax1, ax2) = plt.subplots(figsize=(18, 8), ncols=2)
fig.suptitle("Potential Sites for Targeted Capacity Investments", fontsize=16,weight='bold')
# Set axis off for both subplots
ax1.set_axis_off()
ax2.set_axis_off()

# Shadow effect offset
shadow_offset = 0.01

# Plot solar map on ax1
# Add shadow effect for solar map
boundary.geometry = boundary.geometry.translate(xoff=shadow_offset, yoff=-shadow_offset)
boundary.plot(ax=ax1, color='None', edgecolor='grey', linewidth=1, alpha=0.7)  # Shadow layer
boundary.geometry = boundary.geometry.translate(xoff=-shadow_offset, yoff=shadow_offset)

# Plot solar cells
resource_clusters_solar.plot(column='solar_category', ax=ax1, cmap='Wistia', legend=True, 
           legend_kwds={'title': "Solar Capacity (MW)", 'loc': 'upper right','bbox_to_anchor':(legend_x_ax_offset,1), 'frameon': False})

# Plot actual boundary for solar map
boundary.plot(ax=ax1, facecolor='none', edgecolor='black', linewidth=0.2, alpha=0.7)
"""
# Annotate region numbers for solar map
for idx, row in boundary.iterrows():
    centroid = row.geometry.centroid
    ax1.annotate(f"{row['Region_Number']}", 
                 xy=(centroid.x, centroid.y), 
                 ha='center', va='center',
                 fontsize=7, color='black',
                 bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, boxstyle='round,pad=0.2'))
"""
# Plot wind map on ax2
# Add shadow effect for wind map
boundary.geometry = boundary.geometry.translate(xoff=shadow_offset, yoff=-shadow_offset)
boundary.plot(ax=ax2, color='None', edgecolor='grey', linewidth=1, alpha=0.7)  # Shadow layer
boundary.geometry = boundary.geometry.translate(xoff=-shadow_offset, yoff=shadow_offset)

# Plot wind cells
resource_clusters_wind.plot(column='wind_category', ax=ax2, cmap='summer', legend=True, 
           legend_kwds={'title': "Wind Capacity (MW)", 'bbox_to_anchor':(legend_x_ax_offset,1), 'frameon': False})

# Plot actual boundary for wind map
boundary.plot(ax=ax2, facecolor='none', edgecolor='black', linewidth=0.2, alpha=0.7)
"""
# Annotate region numbers for wind map
for idx, row in boundary.iterrows():
    centroid = row.geometry.centroid
    ax2.annotate(f"{row['Region_Number']}", 
                 xy=(centroid.x, centroid.y), 
                 ha='center', va='center',
                 fontsize=8, color='black',
                 bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, boxstyle='round,pad=0.2'))
"""
# Adjust layout for cleaner appearance
fig.patch.set_alpha(0)  # Make figure background transparent
plt.tight_layout()

# Show the side-by-side plot

# plt.savefig('solar_wind_capacity_map.png',dpi=300)
plt.show()

In [ ]:
resource_clusters_wind.plot(column='potential_capacity',cmap='summer', legend=True,)